In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time


In [7]:

# Path to Chrome driver and profile
driver_path = '/path/to/chromedriver'  # Replace with your chromedriver path
profile_path = r'C:\Users\LENOVO\AppData\Local\Google\Chrome\User Data\Default'  # Replace with your profile path

# YouTube video ID and URL
youtubeid = "W9-racb2rAY"
video_url = f"https://www.youtube.com/watch?v={youtubeid}"

In [8]:


def initialize_driver_with_profile(profile_path):
    """Inisialisasi driver dengan profil Chrome yang sudah ada."""
    chrome_options = Options()
    chrome_options.add_argument(f"user-data-dir={profile_path}")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-first-run")
    chrome_options.add_argument("--remote-debugging-port=9222")
    chrome_options.add_argument("--enable-logging")
    chrome_options.add_argument("--v=1")  # Level logging dasar
    # service = Service(driver_path)
    # driver = webdriver.Chrome(service=service, options=chrome_options)
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def detect_and_skip_ad(driver):
    """Detect and skip ad if present."""
    ad_present = False
    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_any_elements_located([
                (By.CLASS_NAME, "video-ads.ytp-ad-module"),
                (By.CLASS_NAME, "ytp-ad-player-overlay-layout_")
            ])
        )
        ad_present = True
        print("Ad detected.")

        skip_button = WebDriverWait(driver, 30).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "ytp-skip-ad-button"))
        )
        skip_button.click()
        print("Skip Ad button clicked.")

    except Exception as e:
        print("No ad detected or Skip Ad button not found.")
        print(f"Error: {e}")

    return ad_present

def get_all_youtube_comments(driver, max_comments=None):
    """Fetch all comments by scrolling."""
    comments = []
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="content-text"]'))
            )
        except Exception as e:
            print("Comments did not load in time.")
            break

        comment_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
        for element in comment_elements[len(comments):]:
            comment_text = element.text
            if comment_text:
                comments.append(comment_text)
                if max_comments and len(comments) >= max_comments:
                    return comments

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    return comments

def save_to_csv(comments, filename):
    """Save comments to CSV."""
    df = pd.DataFrame(comments, columns=["Comment"])
    df.to_csv(filename, index=False)
    print(f"Comments saved to {filename}")


In [9]:

# Initialize WebDriver with Chrome profile
driver = initialize_driver_with_profile(profile_path)

# Access YouTube video
driver.get(video_url)
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "comments")))

# Detect and skip ad if present
ad_present = detect_and_skip_ad(driver)

# Begin fetching comments
comments = get_all_youtube_comments(driver, max_comments=None)
save_to_csv(comments, filename=f"youtube_comments_{youtubeid}.csv")

# Quit driver
driver.quit()


No ad detected or Skip Ad button not found.
Error: module 'selenium.webdriver.support.expected_conditions' has no attribute 'presence_of_any_elements_located'
Comments saved to youtube_comments_W9-racb2rAY.csv
